In [ ]:
import os
import cv2
import mediapipe as mp
from tqdm import tqdm


input_root = r"C:\Users\user\Desktop\GP dataset\Student engagement"  
output_root = "mesh"  


mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, min_detection_confidence=0.5)


def crop_face(image):
    
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    
    results = face_mesh.process(rgb_image)

    if results.multi_face_landmarks:
        
        landmarks = results.multi_face_landmarks[0]

        
        h, w, _ = image.shape
        min_x = min([landmark.x * w for landmark in landmarks.landmark])
        min_y = min([landmark.y * h for landmark in landmarks.landmark])
        max_x = max([landmark.x * w for landmark in landmarks.landmark])
        max_y = max([landmark.y * h for landmark in landmarks.landmark])

        
        margin = 0.1
        x_exp = int(max(min_x - margin * w, 0))
        y_exp = int(max(min_y - margin * h, 0))
        x2_exp = int(min(max_x + margin * w, w))
        y2_exp = int(min(max_y + margin * h, h))

        
        cropped_face = image[y_exp:y2_exp, x_exp:x2_exp]
        return cropped_face
    return None


for class_name in os.listdir(input_root):
    class_input_path = os.path.join(input_root, class_name)
    class_output_path = os.path.join(output_root, class_name)

    if not os.path.isdir(class_input_path):
        continue
    os.makedirs(class_output_path, exist_ok=True)

    for img_name in tqdm(os.listdir(class_input_path), desc=f"Processing {class_name}"):
        input_path = os.path.join(class_input_path, img_name)
        output_path = os.path.join(class_output_path, img_name)

        img = cv2.imread(input_path)
        if img is None:
            continue

        face = crop_face(img)
        if face is not None:
            face = cv2.resize(face, (224, 224))  
            cv2.imwrite(output_path, face)


Processing Looking Away: 100%|██████████| 423/423 [00:08<00:00, 51.41it/s]
